In [1]:
import pandas as pd
import numpy as np
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F

Data Processing Prepare Training Data Set for DNN solution. 
For the data of 33 person, we chose 30 for training and 3 for validation. 
First Let's parse csvs.

In [2]:
def get_second_data(df):
    df['time'] = pd.to_datetime(df['time'])   # convert 'time' to datetime object
    df['time'] = df['time'].dt.floor("S")
    df['Second'] = df['time'].dt.second
    df['Minute'] = df['time'].dt.minute
    df_second = df.groupby(['Minute','Second']).mean()
    return df_second

In [3]:
loc = ['w','t']
label = ['downstairs','jogging','upstairs','walk_fast','walk_mod','walk_slow']

In [4]:
for loc_i in loc:
    for label_i in label:
        for num in range(26, 64):
            path = './%s/%s/0%g.csv'%(loc_i,label_i,num)
            new_path = './%s/%s/0%g_second.csv'%(loc_i,label_i,num)
            try:
                df = pd.read_csv(path)
                new_df = get_second_data(df)
                new_df.to_csv(new_path)
            except:
                continue

            

In [5]:
def combine_t_w(id, label,label_id):
    w_path = './w/%s/0%g_second.csv'%(label, id)
    t_path = './t/%s/0%g_second.csv'%(label, id)
    df_w = pd.read_csv(w_path)
    df_t = pd.read_csv(t_path)
    df_w = df_w.rename(columns={'x':'w_x','y':'w_y','z':'w_z'})
    df_t = df_t.rename(columns={'x':'t_x','y':'t_y','z':'t_z'})
    df_t_w = pd.merge(df_w,df_t)
    df_t_w['label_id'] = label_id
    return df_t_w

In [6]:
for label_i in label:
    for num in range(26, 64):
        path = './%s/%s/0%g.csv'%(loc_i,label_i,num)
        new_path = './%s/%s/0%g_second.csv'%(loc_i,label_i,num)
        try:
            df = pd.read_csv(path)
            new_df = get_second_data(df)
            new_df.to_csv(new_path)
        except:
            continue

In [9]:
for label_i in label:
    os.makedirs('./combined_data/'+ label_i, exist_ok=True)

In [10]:
column_names = ['Minute','Second','w_x','w_y','w_z','t_x','t_y','t_z']
df = pd.DataFrame(columns = column_names)

In [11]:
def sliding_window(window_length, stride, new_df):
    data = []
    label_id = new_df.iloc[0]['label_id']
    for i in range(0,len(new_df)-window_length,stride):
        start = i
        end = i + window_length
        np_data = np.array([])
        np_data = np.append(new_df.iloc[0:10][['w_x','w_y','w_z','t_x','t_y','t_z']].values.reshape(60,order='F'),label_id)
        data.append(np_data)
    data = np.array(data)
    return data

In [12]:
for num in range(26, 64):
    for i in range(len(label)):
        new_path = './combined_data/%s/0%g.csv'%(label[i],num)
        try:
            new_df = combine_t_w(num, label[i],i)
            window_length = 10
            stride = 1
            if num == 26 and i == 0:
                data_window = sliding_window(window_length, stride, new_df)
            else:
                temp_window = sliding_window(window_length, stride, new_df)
                data_window = np.append(data_window, temp_window,axis=0)
        except:
            continue

In [13]:
# Learn the embedding for each x,y,z
class linear_embedding(nn.Module):
    def __init__(self):
        super(linear_embedding, self).__init__()
        self.layer1 = nn.Linear(10,16)
        self.layer2 = nn.Linear(16,5)
        self.activation = nn.LeakyReLU(0.1)
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.activation(x)
        return x

In [14]:
class deep_model(nn.Module):
    def __init__(self):
        super(deep_model, self).__init__()
        self.le_w_x = linear_embedding()
        self.le_w_y = linear_embedding()
        self.le_w_z = linear_embedding()
        self.le_t_x = linear_embedding()
        self.le_t_y = linear_embedding()
        self.le_t_z = linear_embedding()
        self.layer1 = nn.Linear(30,16)
        self.layer2 = nn.Linear(16,6)
        self.activation = nn.LeakyReLU(0.1)
    def forward(self,x):
        w_x = self.le_w_x(x[:,0:10])
        w_y = self.le_w_y(x[:,10:20])
        w_z = self.le_w_z(x[:,20:30])
        t_x = self.le_t_x(x[:,30:40])
        t_y = self.le_t_y(x[:,40:50])
        t_z = self.le_t_z(x[:,50:60])
        embedding = torch.cat((w_x,w_y,w_z,t_x,t_y,t_z),dim = 1)
        x = self.layer1(embedding)
        x = self.activation(x)
        x = self.layer2(x)
        return x

In [34]:
np.random.shuffle(data_window)
X = torch.Tensor(data_window[:,0:60])
Y = torch.Tensor(data_window[:, 60].astype(int)).type(torch.long)
test_count = 2000
X_test = X[0:test_count]
Y_test = Y[0:test_count]
X_train = X[test_count:]
Y_train = Y[test_count:]

In [35]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, Y_train),batch_size = 16, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, Y_test),batch_size = 16, shuffle=True)

In [36]:
my_model = deep_model()
optimizer = torch.optim.Adam(my_model.parameters(),lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=20000,gamma=0.5)
my_loss = nn.CrossEntropyLoss()

In [37]:
from datetime import datetime
loss = 0
for epoch in range(20):
    my_model.train()
    for x,y in train_loader:
        optimizer.zero_grad()
        output = my_model(x)
        loss = my_loss(output,y)
        loss.backward()
        optimizer.step()
        #print("%s: epoch %g current training loss is %f"%(datetime.now(), epoch, loss))
    scheduler.step()
    my_model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            output = my_model(x)
            _,predicted = torch.max(output.data,1)
            correct += (predicted==y).sum().item()
        print('Epoch %g done, testing accuracy is %f'%(epoch,correct/test_count*100))

Epoch 0 done, testing accuracy is 96.150000
Epoch 1 done, testing accuracy is 100.000000
Epoch 2 done, testing accuracy is 99.400000
Epoch 3 done, testing accuracy is 100.000000
Epoch 4 done, testing accuracy is 100.000000
Epoch 5 done, testing accuracy is 100.000000
Epoch 6 done, testing accuracy is 100.000000
Epoch 7 done, testing accuracy is 100.000000
Epoch 8 done, testing accuracy is 100.000000
Epoch 9 done, testing accuracy is 100.000000
Epoch 10 done, testing accuracy is 100.000000
Epoch 11 done, testing accuracy is 100.000000
Epoch 12 done, testing accuracy is 100.000000
Epoch 13 done, testing accuracy is 100.000000
Epoch 14 done, testing accuracy is 100.000000
Epoch 15 done, testing accuracy is 100.000000
Epoch 16 done, testing accuracy is 95.750000
Epoch 17 done, testing accuracy is 99.500000
Epoch 18 done, testing accuracy is 100.000000
Epoch 19 done, testing accuracy is 100.000000
